In [1]:
from pyspark.sql import SparkSession
#spark is managed by this thing called session- so you wanna build this spark session 1st, so use sparksession.builder and usually you 
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))
devices = spark.read.option("header","true").csv("/home/iceberg/data/devices.csv")
#df.join(df,lit(1) == lit(1)).collect() --this is crossjoin and using collect will give oom problem so 
#instead of collect use .take(5) or show dont wanna pull entire dataset into the driver(badpractice to use .collect())
#if you want to use collect then make sure data in already aggregated down and use it as datasetr

df = events.join(devices,on="device_id",how="left")
df = df.withColumnsRenamed({'browser_type': 'browser_family', 'os_type': 'os_family'})

df.show()



25/07/16 02:21:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| device_id|    user_id|referrer|                host|       url|          event_time|         event_date|browser_family|os_family|device_type|
+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| 532630305| 1037710827|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|         Other|    Other|      Other|
| 532630305|  925588856|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|         Other|    Other|      Other|
| 532630305|-1180485268|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|         Other|    Other|      Other|
| 532630305|-1044833855|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|         Other|    Other|     

In [2]:
#lets see how we are gonna split things up 
#1.repartition(10,col(event_date)- however many partition we had before we now gonna have 10 partitions, and we are split it up  on event date column 
# then we are gonna sort within the partition.using sortWithinPartitions


In [2]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.show()
sortedTwo.show()
#sortWithinPartitions and sort these two api's are different at scale and pulls different data 
#sortWithinPartitions -this will look into 10 partitions you have,and sort all of them locally like within 
#sort- works as global sort and sort back data as far back you can go-slow

+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
|  device_id|    user_id|            referrer|                host|                 url|          event_time|         event_date|browser_family|os_family|       device_type|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
|  532630305| 1129583063|                NULL|admin.zachwilson....|                   /|2021-01-07 09:21:...|2021-01-07 00:00:00|         Other|    Other|             Other|
| 1088283544| -648945006|                NULL|    www.eczachly.com|                   /|2021-01-07 02:58:...|2021-01-07 00:00:00|      PetalBot|  Android|Generic Smartphone|
| -158310583|-1871780024|                NULL|    www.eczachly.com|                   /|2021-01-07 04:17:...|2021-01-07 00:00:00| 

[Stage 9:=============================>                             (4 + 4) / 8]

+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
|  device_id|    user_id|            referrer|                host|                 url|          event_time|         event_date|browser_family|os_family|       device_type|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
| -643696601| 1272828233|                NULL|admin.zachwilson....|                   /|2021-01-02 13:53:...|2021-01-02 00:00:00|        Chrome|  Windows|             Other|
|  532630305|  747494706|                NULL|admin.zachwilson....|                   /|2021-01-02 19:36:...|2021-01-02 00:00:00|         Other|    Other|             Other|
|  898871897| 2110046626|                NULL|admin.zachwilson....|       /wp-login.php|2021-01-02 19:57:...|2021-01-02 00:00:00| 

In [3]:
# .sortWithinPartitions() sorts within partitions, whereas .sort() is a global sort, which is very slow

# Note - exchange is synonymous with Shuffle

In [4]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.explain()
sortedTwo.explain()
#in explain-read from mosrt inteded lowest to up ,project = select in explain, in sort if false present means its not doing global search
#whenever you see Exchange in query plan that means its shuffle -shuffling the data
# if we remove repartition in both sorted and sortedtwo, then sorted will not have even 1 partition, and incase of shuffletwo it will still have 1 partition.
# its beacuse of difference sortWithinPartions and sort
#if you have 20gb data and do the sort and not sortWithinPartition then it wil have to pass all the data to 1 executer to sort the data =slow

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#261, event_date#29, browser_family#74, os_family#75, device_type#58]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=410]
         +- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#56 AS browser_family#74, os_type#57 AS os_family#75, device_type#58]
            +- BroadcastHashJoin [device_id#18], [device_id#55], LeftOuter, BuildRight, false
               :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               :  +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters:

In [1]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

25/08/06 17:45:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [6]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [7]:
%%sql

DROP TABLE IF EXISTS bootcamp.events_sorted

++
||
++
++

In [8]:
#you can partition your data however you want in iceberg

In [9]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));



++
||
++
++

In [ ]:
# Iceberg Table (in Spark):
# Data is stored in files (like Parquet) in cloud storage or HDFS (not a traditional DB server).
# Table is managed by a “table format” (Iceberg) that tracks where all files are, schema, 
# and partitions in a special set of files called a “table metadata
#These files live in cloud (S3, ADLS) or HDFS—not in a database server!
#

In [ ]:
# Diagram: How an Iceberg Table is Organized
# plaintext
# Copy
# Edit
# /bootcamp/events/
# │
# ├── metadata/
# │   └── snap-12345.metadata.json   ← Table metadata/manifest
# │
# ├── year=2022/
# │   └── part-0001.parquet          ← Parquet file for year 2022
# │
# └── year=2023/
#     └── part-0002.parquet          ← Parquet file for year 2023

#    Each partition folder holds data files for that year.

# Metadata tracks ALL files, schema, snapshots, partitioning.
# How Does Spark Use Iceberg?
# When you read:
# Spark looks at the metadata, finds the files for your query (e.g., only year=2023), reads only those files, and returns the result as a DataFrame.
# When you write:
# Spark writes new Parquet files, updates the metadata with new files/partitions/snapshots.

In [10]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));

++
||
++
++

In [11]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (year(event_date));

++
||
++
++

In [12]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("host"))

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [13]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
5444946,4,sorted
5556664,4,unsorted


In [14]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
None,0


In [16]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bootcamp`.`matches_bucketed`.`files` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 21;
'Aggregate [unresolvedalias(count(1), None)]
+- 'UnresolvedRelation [bootcamp, matches_bucketed, files], [], false


In [40]:

df1 = events.join(devices,on="device_id",how="left")
df1 = df1.withColumnsRenamed({'browser_type': 'browser_family', 'os_type': 'os_family','device_type':'device_family'})

df1.show()

+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-------------+
| device_id|    user_id|referrer|                host|       url|          event_time|         event_date|browser_family|os_family|device_family|
+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-------------+
| 532630305| 1037710827|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|         Other|    Other|        Other|
| 532630305|  925588856|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|         Other|    Other|        Other|
| 532630305|-1180485268|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|         Other|    Other|        Other|
| 532630305|-1044833855|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|         Other|   

In [41]:
df1.columns

['device_id',
 'user_id',
 'referrer',
 'host',
 'url',
 'event_time',
 'event_date',
 'browser_family',
 'os_family',
 'device_family']

In [15]:
#df1=spark.read.option("header","true").csv("/home/iceberg/data/events.csv").withColumn("event_date",expr("DATE_TRUNC('day',event_time)"))

In [42]:
start_df1=df1.repartition(4,col("event_date")).withColumn("event_time",col("event_time").cast("timestamp")) 

In [43]:
start_df1.columns

['device_id',
 'user_id',
 'referrer',
 'host',
 'url',
 'event_time',
 'event_date',
 'browser_family',
 'os_family',
 'device_family']

In [75]:
first_sort_df1=start_df1.sortWithinPartitions(col("event_date"),col("browser_family"),col("host"))

In [76]:
start_df1.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted_v2")


In [77]:
%%sql
select * from bootcamp.events_unsorted_v2 limit 10

device_id,user_id,referrer,host,url,event_time,event_date,browser_family,os_family,device_family
532630305,747494706,None,www.zachwilson.tech,/,2021-09-26 16:03:17.535000,2021-09-26 00:00:00,Other,Other,Other
532630305,747494706,None,admin.zachwilson.tech,/,2021-02-21 16:08:17.975000,2021-02-21 00:00:00,Other,Other,Other
532630305,-824540328,None,admin.zachwilson.tech,/,2021-09-28 17:23:14.992000,2021-09-28 00:00:00,Other,Other,Other
-733097781,5890109,None,www.zachwilson.tech,/,2023-01-09 11:55:28.032000,2023-01-09 00:00:00,curl,Other,Other
532630305,-707954698,None,admin.zachwilson.tech,/,2021-06-19 05:55:00.559000,2021-06-19 00:00:00,Other,Other,Other
532630305,747494706,None,admin.zachwilson.tech,/,2021-06-19 15:09:19.519000,2021-06-19 00:00:00,Other,Other,Other
-733097781,5890109,None,www.zachwilson.tech,/,2023-01-09 20:10:27.610000,2023-01-09 00:00:00,curl,Other,Other
532630305,696863716,None,admin.zachwilson.tech,/,2023-01-10 04:43:49.204000,2023-01-10 00:00:00,Other,Other,Other
532630305,-1180485268,None,www.eczachly.com,/,2021-06-28 16:51:11.679000,2021-06-28 00:00:00,Other,Other,Other
532630305,-1180485268,None,www.zachwilson.tech,/,2021-06-28 17:34:44.423000,2021-06-28 00:00:00,Other,Other,Other


In [78]:
first_sort_df1.write.mode("overwrite").saveAsTable("bootcamp.events_sorted_v2")

In [79]:
%%sql
select * from bootcamp.events_sorted_v2 limit 10

device_id,user_id,referrer,host,url,event_time,event_date,browser_family,os_family,device_family
589185851,-21136712,None,admin.zachwilson.tech,/,2021-01-12 18:49:28.425000,2021-01-12 00:00:00,Chrome,Linux,Other
589185851,-414920062,None,admin.zachwilson.tech,/,2021-01-12 18:54:30.995000,2021-01-12 00:00:00,Chrome,Linux,Other
589185851,-414920062,None,admin.zachwilson.tech,/,2021-01-12 19:56:56.809000,2021-01-12 00:00:00,Chrome,Linux,Other
589185851,-694958230,None,admin.zachwilson.tech,/,2021-01-12 20:08:15.964000,2021-01-12 00:00:00,Chrome,Linux,Other
-843023486,-2116612468,None,www.eczachly.com,/,2021-01-12 01:13:53.762000,2021-01-12 00:00:00,Chrome,Mac OS X,Other
-843023486,-2116612468,https://www.eczachly.com/,www.eczachly.com,/blog,2021-01-12 01:13:56.914000,2021-01-12 00:00:00,Chrome,Mac OS X,Other
-843023486,-2116612468,https://www.eczachly.com/blog,www.eczachly.com,/graphs,2021-01-12 01:13:58.899000,2021-01-12 00:00:00,Chrome,Mac OS X,Other
-843023486,-2116612468,https://www.eczachly.com/graphs,www.eczachly.com,/graphs,2021-01-12 01:14:01.017000,2021-01-12 00:00:00,Chrome,Mac OS X,Other
-843023486,-2116612468,https://www.eczachly.com/graphs,www.eczachly.com,/about,2021-01-12 01:14:02.615000,2021-01-12 00:00:00,Chrome,Mac OS X,Other
-843023486,-2116612468,https://www.eczachly.com/about,www.eczachly.com,/contact,2021-01-12 01:14:40.193000,2021-01-12 00:00:00,Chrome,Mac OS X,Other


In [80]:
%%sql

select sum(file_size_in_bytes) as  size, count(1) as num_files from demo.bootcamp.events_sorted_v2.files
UNION ALL
select sum(file_size_in_bytes) as size, count(1) as num_files from demo.bootcamp.events_unsorted_v2.files

size,num_files
5095130,4
5556684,4


In [ ]:
# you see if we only use date for sorting its helped only tiny bit , 
# you can see the diff between sorted and unsorted file size,and if you use cols ,col("event_date"),col("browser_family"),col("host") ther
# is big difference between sorted and unsorted and sorted records are more less size when you choose correct columns while sorting(play around),you get
# better comression.
# so when you are writing your data out ,you want to be your data written out from yoiur lowest cardinality 1st all the way down
# to your highest cardinality when you are sorting data, in this case event_date is lowest as there are only couple of 100 records there.
# you can shrink your data down this way

In [1]:
python version

SyntaxError: invalid syntax (1241365062.py, line 1)